In [103]:
import requests
import asyncio
from aiohttp import ClientSession

import pandas as pd

### Helpers

In [2]:
# Dont do this, create a new session per context instead
# aioSession = ClientSession()

In [3]:
# https://stackoverflow.com/questions/48840378/python-attempt-to-decode-json-with-unexpected-mimetype
async def fetch(url, session, method='GET'):
    try:
        response = await session.request(method=method, url=url)
        response.raise_for_status()
        return await response.json(content_type=None)
    except Exception as err:
        print(f"An error ocurred: {err}")

### URLs

In [4]:
base_url = 'https://resultados.onpe.gob.pe/v1'

organizaciones_politicas_url = f'{base_url}/IN2020/agpcand/D40015'
tipo_elecciones_partido_url = f'{base_url}/IN2020/elecciones'

resumen_partido_url = f'{base_url}/IN2020/summary/actas'

resultados_elecciones_presidencialtes_url = f'{base_url}/IN2020/results/10/800000'
resultados_elecciones_parlamento_andino_url = f'{base_url}/IN2020/results/12/800000'
resultados_elecciones_congresales_url = f'{base_url}/IN2020/results/11/D43015'
# resultados_elecciones_congresales_por_distrito_electoral_url = f'{base_url}/IN2020/results/11/D43000/'


ubigeos_regionales_por_partido_url = f'{base_url}/DE2020/ubigeos/01'
ubigeos_provinciales_por_partido_url = f'{base_url}/DE2020/ubigeos/03'
ubigeos_distritales_por_partido_url = f'{base_url}/DE2020/ubigeos/04'
resultados_elecciones_delegados_regional_url = f'{base_url}/DE2020/results/01'
resultados_elecciones_delegados_provincial_url = f'{base_url}/DE2020/results/03'
resultados_elecciones_delegados_distritales_url = f'{base_url}/DE2020/results/04'

iconos_organizaciones_url = 'https://resultados.onpe.gob.pe/IN2020/assets/iconos/iconos_reales'

---
## Consulta de Datos

### Lista de organizaciones políticas

In [5]:
def parse_org_politicas(org):
    org_parsed = dict(
        organizacion=org['C_DESC_AGRUPOL'], 
        codigo=org['C_CODI_AGRUPOL'],
        codigo_corto=org['C_CODI_AGRUPOL'][-2:]
    )
    return org_parsed

def fetch_org_politicas():
    organizaciones = requests.get(organizaciones_politicas_url).json()['organizaciones']
    organizaciones_parsed = []
    for org in organizaciones:
        organizaciones_parsed.append(parse_org_politicas(org))
    return organizaciones_parsed

In [6]:
organizaciones_politicas = fetch_org_politicas()

In [7]:
organizaciones_politicas[0]

{'organizacion': 'ACCION POPULAR', 'codigo': '00000002', 'codigo_corto': '02'}

### Detalles básicos

In [8]:
def clean_resumen_general(resumen):
    resumen_parsed = dict(
        electores_habiles=int(resumen['n_elec_habil'].replace(',', '')),
        cantidad_mesas=int(resumen['n_num_mesas']),
        cantidad_locales=int(resumen['locales']),
    )
    return resumen_parsed

async def fetch_org_base_detail(org):
    async with ClientSession() as session:
        tipo_elecciones_url = f'{tipo_elecciones_partido_url}/{org["codigo"]}'
        resumen_general_url = f'{resumen_partido_url}/{org["codigo_corto"]}'
        urls = [tipo_elecciones_url, resumen_general_url]
        tipo_elecciones, resumen_general = await asyncio.gather(*[fetch(url, session) for url in urls])

        org['tipo_elecciones'] = tipo_elecciones['elecciones']
        org['resumen_general'] = clean_resumen_general(resumen_general)

        return org
    
async def fetch_all_base_details():
    return await asyncio.gather(
        *[fetch_org_base_detail(org) for org in organizaciones_politicas]
    )

In [9]:
org_detalles_basicos = await fetch_all_base_details()

In [10]:
org_detalles_basicos[0]

{'organizacion': 'ACCION POPULAR',
 'codigo': '00000002',
 'codigo_corto': '02',
 'tipo_elecciones': ['10', '12', '11'],
 'resumen_general': {'electores_habiles': 221946,
  'cantidad_mesas': 326,
  'cantidad_locales': 127}}

### Resultados elecciones por organización política

In [11]:
def get_general_data_results(resultados):
    general_data = resultados['generals']['generalData']
    relevant_data = {}
    if general_data:
        relevant_data['ELECTORES_HABIL'] = int(general_data['ELECTORES_HABIL'].replace(',', ''))
        relevant_data['TOT_CIUDADANOS_VOTARON'] = int(general_data['TOT_CIUDADANOS_VOTARON'].replace(',', ''))
        relevant_data['POR_PROCESAR'] = int(general_data['POR_PROCESAR'].replace(',', ''))
    return relevant_data

#### Elecciones por Delegados

In [12]:
def get_eleccion_delegados_params(tipo_eleccion):
    tipo = ''
    resultados_url = ''
    ubigeos_url = ''

    region_level = ''
    region_level_code = ''
    region_level_descripcion = ''

    if tipo_eleccion == '01':
        tipo = 'delegados_regional'
        resultados_url = resultados_elecciones_delegados_regional_url
        ubigeos_url = ubigeos_regionales_por_partido_url
        region_level = 'departments'
        region_level_code = 'CDGO_DEP'
        region_level_descripcion = 'DESC_DEP'
    elif tipo_eleccion == '03':
        tipo = 'delegados_provincial'
        resultados_url = resultados_elecciones_delegados_provincial_url
        ubigeos_url = ubigeos_provinciales_por_partido_url
        region_level = 'provinces'
        region_level_code = 'CDGO_PROV'
        region_level_descripcion = 'DESC_PROV'
    elif tipo_eleccion == '04':
        tipo = 'delegados_distritales'
        resultados_url = resultados_elecciones_delegados_distritales_url
        ubigeos_url = ubigeos_distritales_por_partido_url
        region_level = 'districts'
        region_level_code = 'CDGO_DIST'
        region_level_descripcion = 'DESC_DIST'
    else:
        print('error al consultar tipo de eleccion no hay data')

    return (
        tipo,
        resultados_url,
        ubigeos_url,
        region_level,
        region_level_code,
        region_level_descripcion
    )

In [13]:
def get_eleccion_delegados_resultados_rollup(resultados):
    electores_habiles = 0
    electores_participaron = 0
    por_procesar = 0
    for resultado in resultados:
        electores_habiles += resultado['resultados']['ELECTORES_HABIL']
        electores_participaron += resultado['resultados']['TOT_CIUDADANOS_VOTARON']
        por_procesar += resultado['resultados']['POR_PROCESAR']

    resultados_rollup = dict(
        ELECTORES_HABIL=electores_habiles,
        TOT_CIUDADANOS_VOTARON=electores_participaron,
        POR_PROCESAR=por_procesar
    )
    
    return resultados_rollup

In [14]:
async def eleccion_delegados_results_by_ubigeo(
    org, level_ubigeo, resultados_url, region_level_code, region_level_descripcion
):
    async with ClientSession() as session:
        ubigeo_results_url = f'{resultados_url}/{level_ubigeo[region_level_code]}/{org["codigo"]}'
        resultados_por_nivel_region = await fetch(ubigeo_results_url, session)

        general_data = get_general_data_results(resultados_por_nivel_region)

        nivel_region_detalle = dict(
            nombre=level_ubigeo[region_level_descripcion],
            codigo=level_ubigeo[region_level_code],
            codigo_padre=level_ubigeo['CDGO_PADRE'],
            resultados=general_data
        )

        return nivel_region_detalle

In [15]:
async def process_eleccion_delegados(org, tipo_eleccion):
    (tipo,
    resultados_url,
    ubigeos_url,
    region_level,
    region_level_code,
    region_level_descripcion) = get_eleccion_delegados_params(tipo_eleccion)

    # fetch ubigeos
    ubigeos_list_url = f'{ubigeos_url}/{org["codigo"]}'
    async with ClientSession() as session:
        ubigeos = await fetch(ubigeos_list_url, session)

    # fetch results by ubigeo
    results_by_ubigeo = []
    for level_ubigeo in ubigeos[region_level]:
        ubigeo_result = eleccion_delegados_results_by_ubigeo(
            org, level_ubigeo, resultados_url, region_level_code, region_level_descripcion
        )
        results_by_ubigeo.append(ubigeo_result)
    resultados = await asyncio.gather(*results_by_ubigeo)
    
    resultados_rollup = get_eleccion_delegados_resultados_rollup(resultados)

    return dict(
        resultados=resultados_rollup, 
#         resultados_rollup=resultados_rollup, 
        tipo=tipo, region_level=region_level
    )

In [16]:
await process_eleccion_delegados(org_detalles_basicos[1], '01')

{'resultados': {'ELECTORES_HABIL': 224604,
  'TOT_CIUDADANOS_VOTARON': 4807,
  'POR_PROCESAR': 0},
 'tipo': 'delegados_regional',
 'region_level': 'departments'}

#### Elecciones Universales (1 militante 1 voto)

In [17]:
def get_eleccion_por_elector_params(tipo_eleccion):
    tipo = ''
    url = ''
    if tipo_eleccion == '10':
        tipo = 'presidenciales'
        url = resultados_elecciones_presidencialtes_url
    elif tipo_eleccion == '11':
        tipo = 'congresales'
        url = resultados_elecciones_congresales_url
    elif tipo_eleccion == '12':
        tipo = 'parlamento_andino'
        url = resultados_elecciones_parlamento_andino_url
        
    return (tipo, url)

In [18]:
async def process_eleccion_por_elector(org, tipo_eleccion):
    (tipo, url) = get_eleccion_por_elector_params(tipo_eleccion)
    
    async with ClientSession() as session:
        results_url = f'{url}/{org["codigo"]}'
        resultados = await fetch(results_url, session)

        general_data = get_general_data_results(resultados)
        
        return dict(resultados=general_data, tipo=tipo)

In [19]:
await process_eleccion_por_elector(org_detalles_basicos[0], '10')

{'resultados': {'ELECTORES_HABIL': 221946,
  'TOT_CIUDADANOS_VOTARON': 30413,
  'POR_PROCESAR': 0},
 'tipo': 'presidenciales'}

#### Resultados agregados según el tipo de elección

In [20]:
async def process_org_politica_results(org):
    resultados_por_tipo = []
    for tipo_eleccion in org['tipo_elecciones']:
        elecciones_delegados = ['01', '03', '04']
        process_type = None
        if tipo_eleccion in elecciones_delegados:
            process_type = process_eleccion_delegados
        else:
            process_type = process_eleccion_por_elector
        resultados_por_tipo.append(process_type(org, tipo_eleccion))

    resultados = await asyncio.gather(*resultados_por_tipo)
    org['resultados_tipo_elecciones'] = resultados
    return org

In [21]:
await process_org_politica_results(org_detalles_basicos[2])

{'organizacion': 'AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL',
 'codigo': '00000016',
 'codigo_corto': '16',
 'tipo_elecciones': ['01', '12', '11', '10'],
 'resumen_general': {'electores_habiles': 7881,
  'cantidad_mesas': 31,
  'cantidad_locales': 29},
 'resultados_tipo_elecciones': [{'resultados': {'ELECTORES_HABIL': 7881,
    'TOT_CIUDADANOS_VOTARON': 362,
    'POR_PROCESAR': 0},
   'tipo': 'delegados_regional',
   'region_level': 'departments'},
  {'resultados': {'ELECTORES_HABIL': 50,
    'TOT_CIUDADANOS_VOTARON': 39,
    'POR_PROCESAR': 0},
   'tipo': 'parlamento_andino'},
  {'resultados': {}, 'tipo': 'congresales'},
  {'resultados': {'ELECTORES_HABIL': 50,
    'TOT_CIUDADANOS_VOTARON': 39,
    'POR_PROCESAR': 0},
   'tipo': 'presidenciales'}]}

### Resultados Generales

In [22]:
async def fetch_all_results():
    return await asyncio.gather(
        *[process_org_politica_results(org) for org in org_detalles_basicos]
    )

In [ ]:
resultados_generales = await fetch_all_results()

---

## Parseo y guardado

### Tipo de elecciones por organización política

In [106]:
voto_delegados = [
    'ALIANZA PARA EL PROGRESO',
    'AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL',
    'FRENTE POPULAR AGRICOLA FIA DEL PERU - FREPAP', 'FUERZA POPULAR',
    'PARTIDO FRENTE DE LA ESPERANZA 2021', 'PARTIDO POLITICO CONTIGO',
    'PARTIDO POLITICO NACIONAL PERU LIBRE',
    'PERU NACION', 'PERU PATRIA SEGURA', 'PODEMOS PERU',
    'RENACIMIENTO UNIDO NACIONAL', 'RESTAURACION NACIONAL',
    'SOLIDARIDAD NACIONAL', 'TODOS POR EL PERU', 'UNION POR EL PERU'
]

voto_universal = [
    'ACCION POPULAR',
    'EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD',
    'JUNTOS POR EL PERU', 'PARTIDO APRISTA PERUANO',
    'PARTIDO DEMOCRATICO SOMOS PERU', 'PARTIDO MORADO',
    'PARTIDO NACIONALISTA PERUANO',
]

voto_mixto = ['PARTIDO POPULAR CRISTIANO - PPC', 'DEMOCRACIA DIRECTA']

In [107]:
def get_modalidad_y_listas(org, tipo_eleccion):

    modalidad = '-'
    listas = '-'

    # Modalidad
    if org in voto_mixto:
        if org == 'PARTIDO POPULAR CRISTIANO - PPC':
            if tipo_eleccion == 'presidenciales':
                modalidad = 'universal'
            else:
                modalidad = 'delegados'
        elif org == 'DEMOCRACIA DIRECTA':
            if tipo_eleccion == 'congresales':
                modalidad = 'universal'
            else:
                modalidad = 'delegados'
    elif org in voto_universal:
            modalidad = 'universal'
    else:
        modalidad = 'delegados'

    # Numero de listas
    if tipo_eleccion == 'presidenciales':
        if org in ['PARTIDO MORADO', 'PARTIDO APRISTA PERUANO']:
            listas = 3
        elif org == 'ACCION POPULAR':
            listas = 4
        else:
            listas = 1
    else:
        listas = '-'
        
    return (modalidad, listas)

### Data agregada

In [111]:
def resultados_por_organizacion(org):
    resultados = []
    for result in org['resultados_tipo_elecciones']:
        (modalidad, listas) = get_modalidad_y_listas(org['organizacion'], result['tipo'])
        result_clean = dict(
            # detalles generales
            organizacion = org['organizacion'],
            codigo = org['codigo'],
            codigo_corto = org['codigo_corto'],
            total_electores_habiles = org['resumen_general']['electores_habiles'],
            # detalles por tipo de eleccion
            TIPO_ELECCION = result['tipo'],
            ELECTORES_HABILES = result['resultados'].get('ELECTORES_HABIL', None),
            ELECTORES_VOTARON = result['resultados'].get('TOT_CIUDADANOS_VOTARON', None),
            VOTOS_POR_PROCESAR = result['resultados'].get('POR_PROCESAR', None),
            MODALIDAD = modalidad,
            LISTAS = listas,
        )
        resultados.append(result_clean)
    return resultados

In [115]:
def clean_resultados(resultados):
    total_resultados = []
    for org in resultados:
        resultados_org = resultados_por_organizacion(org)
        total_resultados.extend(resultados_org)
    return total_resultados

In [116]:
resultados_clean = clean_resultados(resultados_generales)

In [117]:
df = pd.DataFrame(resultados_clean)

In [133]:
df.head()

,organizacion,codigo,codigo_corto,total_electores_habiles,TIPO_ELECCION,ELECTORES_HABIL,TOT_CIUDADANOS_VOTARON,POR_PROCESAR,MODALIDAD,LISTAS
0,ACCION POPULAR,00000002,02,221946,presidenciales,221946.0,30413.0,0.0,universal,4
1,ACCION POPULAR,00000002,02,221946,parlamento_andino,221946.0,30814.0,0.0,universal,-
2,ACCION POPULAR,00000002,02,221946,congresales,62884.0,9996.0,0.0,universal,-
3,ALIANZA PARA EL PROGRESO,00000012,12,224604,delegados_regional,224604.0,4807.0,0.0,delegados,-
4,ALIANZA PARA EL PROGRESO,00000012,12,224604,parlamento_andino,26.0,26.0,0.0,delegados,-


### Guardado csv

In [ ]:
df.to_csv('elecciones_internas_org_politicas_peru_2020.csv', index=False)

### Fuentes
- [ONPE - Elecciones Internas 2020](https://resultados.onpe.gob.pe/IN2020/Home)